# Chapter 17. Future를 이용한 동시성

이 장에서는 `concurrent.futures` 라이브러리를 중점적으로 알아본다. 그리고 비동기 작업의 실행을 나타내는 객체인 `Future`의 개념에 대해 소개한다.

## 17.1 예제: 세 가지 스타일의 웹 내려받기

긴 지연 시간동안 네트워크 입출력을 효율적으로 처리하려면 동시성을 이용해야 한다. 즉, 네트워크에서 응답이 오는 동안 다른 일을 처리하는 것이 좋다. 여기서는 웹에서 20개 국가의 국기 이미지를 내려받는 프로그램을 3개 작성한다. 

1. `flags.py`는 순차적으로 실행되므로 이전 이미지를 내려받아 디스크에 저장한 후 다음번 이미지를 내려받는다. 

2. `flags_threadpool.py`는 `concurrent.futures` 패키지를 사용하여 모든 이미지를 동시에 요청한 후 도착하는 대로 파일에 저장한다.

3. `flags_asyncio.py`는 `asyncio`를 이용하여 동시에 저장한다.

여기서 동시성 스크립트들은 순차적인 스크립트보다 5배 정도 빠르다. 3번은 18장에서 소개한다.

### 17.1.1 순차 내려받기 스크립트

In [7]:
# flags.py

import os
import time
import sys

import requests
# 표준 라이브러리에 속해 있지 않으므로 한줄 띄고 임포트

# 국기 사이트가 뻗었으므로 걍 내 이미지로 하자...
POP20_CC = ('8_1 8_2 8_3 8_4 8_5 '
           '13_1 13_2 13_3 15_1').split()

#BASE_URL = 'http://flupy.org/data/flags'
BASE_URL = 'https://github.com/lih0905/Fluent_Python/raw/master/images/'

# 이미지 저장할 디렉토리 (미리 만들어놔야함)
DEST_DIR = 'downloads/'

# 이미지(바이트 시퀀스) 저장
def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)

# 이미지 이름을 인수로 받아서 URL을 만들고 이미지(이진 시퀀스)를 내려받는다.
def get_flag(cc):
    url = '{}/{cc}.PNG'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content

# 문자열을 출력하고 sys.stdout.flush를 호출해서 진행 상황을 한 줄에 출력한다.
# 일반적으로 파이썬은 개행 문자를 받기 전까지 문자열을 출력하지 않으므로 
# sys.stdout.flush()를 호출해서 stdout 버퍼에 남아 있는 내용을 모두 화면에 출력하게 해야한다.
def show(text):
    print(text, end=' ')
    sys.stdout.flush()

# 동시성 버전과 다른 핵심 부분
def download_many(cc_list):
    for cc in sorted(cc_list): # 출력할 때 순서대로 나오게 만든다.
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.PNG')
    return len(cc_list)

# download_many()를 실행하는 데 걸린 시간을 기록하고 출력한다.
def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))
    
if __name__ == '__main__':
    main(download_many)

13_1 13_2 13_3 15_1 8_1 8_2 8_3 8_4 8_5 
9 flags downloaded in 5.44s


In [4]:
!python flags.py

13_1 13_2 13_3 15_1 8_1 8_2 8_3 8_4 8_5 
9 flags downloaded in 2.88s


In [5]:
!python flags.py

13_1 13_2 13_3 15_1 8_1 8_2 8_3 8_4 8_5 
9 flags downloaded in 4.01s


In [6]:
!python flags.py

13_1 13_2 13_3 15_1 8_1 8_2 8_3 8_4 8_5 
9 flags downloaded in 2.86s


### 17.1.2 concurrent.futures로 내려받기

`concurrent.futures` 패키지의 가장 큰 특징은 `ThreadPoolExecutor`와 `ProcessPoolExecutor` 클래스인데, 이 클래스들은 콜러블 객체를 서로 다른 스레드나 프로세스에서 실행할 수 있게 해주는 인터페이스를 구현한다.

In [ ]:
# flags_threadpool.py

from concurrent import futures

# flags 모듈의 함수들 재사용
from flags import save_flag, get_flag, show, main

# ThreadPoolExecutor에서 사용할 최대 스레드 수
MAX_WORKERS = 20

# 하나의 이미지를 내려받을 함수
def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.PNG')
    return cc

def download_many(cc_list):
    # 작업자 스레드 수
    workers = min(MAX_WORKERS, len(cc_list))
    # ThreadPoolExecutor 객체 생성
    with futures.ThreadPoolExecutor(workers) as executor:
        # map 메서드는 여러 스레드에 의해 download_one() 함수를 동시에 호출한다
        # 내장된 map() 함수와 비슷하며, 각 함수가 반환한 값을 담은 제너레이터 반환
        res = executor.map(download_one, sorted(cc_list))
    
    # 결과 반환. 호출한 함수중 하나라도 예외를 발생시키면 여기서 에러 발생
    return len(list(res))

if __name__ == '__main__':
    main(download_many)

8_3 8_5 8_4 8_2 8_1 15_1 13_3 13_2 13_1 

이유는 모르겠으나 에러가 남... 빠르긴 하다.

### 17.1.3 Future는 어디에 있나?

Futre는 concurrent.futures와 asyncio의 내부에 있는 핵심 컴포넌트인데, 이 라이브러의 사용자에게 드러나지 않는 경우가 종종 있다. 여기서는 전반적인 Future의 특징에 대해 설명한다. Future 클래스는 완료되었을 수도 있고 아닐 수도 있는 지연된 계산을 표현하기 위해 사용된다. 대기 중인 작업을 큐에 넣고, 완료 상태를 조사하고, 결과를 가져올 수 있도록 캡슐화 한다.

In [ ]:
# flags_threadpool_ac.py

from concurrent import futures

# flags 모듈의 함수들 재사용
from flags import save_flag, get_flag, show, main

# ThreadPoolExecutor에서 사용할 최대 스레드 수
MAX_WORKERS = 20

# 하나의 이미지를 내려받을 함수
def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.PNG')
    return cc

def download_many(cc_list):
    cc_list= cc_list[:5] # 5개만 쓰자
    # 대기중인 Future 객체를 출력해서 보기 위해 max_worker=3 으로 하드코딩
    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            # executor.submit()은 콜러블이 실행되도록 스케줄링하고
            # 이 작업을 나타내는 Future 객체를 반환
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))
        
        results = []
        # as_completed()는 Future가 완료해될 때 해당 Future 객체 생성
        for future in futures.as_completed(to_do):
            res = future.result() # 결과 생성
            msg = '{} result: {!r}'
            print(msg.format(future, res))
            results.append(res)

    return len(list(res))

if __name__ == '__main__':
    main(download_many)

Scheduled for 8_1: <Future at 0x7f586bd415c0 state=running>
Scheduled for 8_2: <Future at 0x7f586bd57c18 state=running>
Scheduled for 8_3: <Future at 0x7f586bd57be0 state=running>
Scheduled for 8_4: <Future at 0x7f586bd7f7b8 state=pending>
Scheduled for 8_5: <Future at 0x7f586bd7f860 state=pending>
8_3 <Future at 0x7f586bd57be0 state=finished returned str> result: '8_3'
8_1 <Future at 0x7f586bd415c0 state=finished returned str> result: '8_1'
8_2 <Future at 0x7f586bd57c18 state=finished returned str> result: '8_2'
8_5 <Future at 0x7f586bd7f860 state=finished returned str> result: '8_5'
8_4 

사실 지금까지 테스트한 동시성 스크립트는 어느 것도 병렬로 내려받을 수 없다. concurrent.futures는 전역 인터프리터 락(GIL)에 의해 제한되며, flags_asyncio.py는 단일 스레드로 실행된다. 따라서 다음 질문이 떠오른다.

* 파이썬 스레드가 한 번에 한 스레드만 실행할 수 있게 해주는 GIL에 의해 제한된다면, 어떻게 flags_threadpool.py가 flags.py보다 5배나 빨리 실행될까?
* 둘다 단일 스레드인데, 어떻게 flags_asyncio.py가 flags.py보다 5배나 빨리 실행될 수 있을까?

In [7]:
!python flags_processpool.py

15_1 8_1 8_4 8_3 8_5 8_2 13_2 13_3 13_1 
9 flags downloaded in 5.64s


In [8]:
!python flags_processpool_ac.py

Scheduled for 8_1: <Future at 0x7ff16cc61f60 state=running>
Scheduled for 8_2: <Future at 0x7ff16c9f0588 state=pending>
Scheduled for 8_3: <Future at 0x7ff16c9f0390 state=pending>
Scheduled for 8_4: <Future at 0x7ff16c9f0710 state=pending>
Scheduled for 8_5: <Future at 0x7ff16c9f07b8 state=running>
8_1 <Future at 0x7ff16cc61f60 state=finished returned str> result: '8_1'
8_3 <Future at 0x7ff16c9f0390 state=finished returned str> result: '8_3'
8_5 <Future at 0x7ff16c9f07b8 state=finished returned str> result: '8_5'
8_2 <Future at 0x7ff16c9f0588 state=finished returned str> result: '8_2'
8_4 <Future at 0x7ff16c9f0710 state=finished returned str> result: '8_4'

3 flags downloaded in 0.16s


## 17.2 블로킹 I/O와 GIL

CPython 인터프리터는 전역 인터프리터 락(GIL)을 통해 한 번에 한 스레드만 파이썬 바이트코드를 실행하도록 제한한다. 따라서 단일 파이썬 프로세스가 동시에 다중 CPU 코어를 사용할 수 없다.

그런데 블로킹 입출력을 실행하는 모든 표준 라이브러리 함수는 OS에서 결과를 기다리는 동안 GIL을 해제한다. 파이썬 스레드가 네트워크로부터의 응답을 기다리는 동안 블로킹된 입출력 함수가 GIL을 해제함으로써 다른 스레드가 실행될 수 있다.

## 17.3 concurrent.futures로 프로세스 실행하기

ProcessPoolExecutor 클래스는 GIL을 우회하여 계산 위주의 작업을 수행하는 경우 가용한 CPU를 모두 사용한다. ProcessPoolExecutor와 ThreadPoolExecutor는 모두 범용 Executor 인터페이스를 구현하므로, concurrent.futures를 사용하는 경우에는 스레드 기반의 프로그램을 프로세스 기반의 프로그램으로 쉽게 변환할 수 있다.

국기 내려받는 프로그램처럼 입출력 위주의 작업에서는 ProcessPoolExecutor를 사용해도 큰 도움이 안된다. 그러나 계산 위주의 작업에서 진가를 발휘한다.

> 그런데 ProcessPoolExecutor 쓰면 오류는 안뜸!

## 17.4 Executor.map() 실험

Executor.map() 메서드를 이용하면 여러 콜러블을 간단히 동시에 실행할 수 있다. 다음 예제는 Executor.map()이 동작하는 과정을 자세히 보여준다.

In [9]:
# demo_executor.map.py

from time import sleep, strftime
from concurrent import futures

def display(*args): # 자신이 받은 인수 앞에 시간 출력
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)
    
# 시작할 때 메시지를 출력하고, 인수로 받은 n초 동안 잠자고, 마지막 메시지 출력.
def loiter(n): 
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n))
    return n * 10

def main():
    display('Script starting.')
    executor = futures.ThreadPoolExecutor(max_workers=3)
    # worker가 3개이므로 일단은 loiter(0), loiter(1), loiter(2)만 먼저 실행
    results = executor.map(loiter, range(5)) 
    display('results:', results)
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))
    
#main()

In [11]:
loiter(1)

[17:26:27] 	loiter(1): doing nothing for 1s...
[17:26:28] 	loiter(1): done.


10

In [1]:
!python demo_executor_map.py

[19:38:56] Script starting.
[19:38:56] loiter(0): doing nothing for 0s...
[19:38:56] loiter(0): done.
[19:38:56] 	loiter(1): doing nothing for 1s...
[19:38:56] 		loiter(2): doing nothing for 2s...
[19:38:56] results: <generator object Executor.map.<locals>.result_iterator at 0x7f22bdb0e390>
[19:38:56] 			loiter(3): doing nothing for 3s...
[19:38:56] Waiting for individual results:
[19:38:56] result 0: 0
[19:38:57] 	loiter(1): done.
[19:38:57] 				loiter(4): doing nothing for 4s...
[19:38:57] result 1: 10
[19:38:58] 		loiter(2): done.
[19:38:58] result 2: 20
[19:38:59] 			loiter(3): done.
[19:38:59] result 3: 30
[19:39:01] 				loiter(4): done.
[19:39:01] result 4: 40
Segmentation fault (core dumped)


무슨 말인지 모르겠다...

In [14]:
!cat demo_executor_map_pr.py

# demo_executor.map.py

from time import sleep, strftime
from concurrent import futures

def display(*args): # 자신이 받은 인수 앞에 시간 출력
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)
    
# 시작할 때 메시지를 출력하고, 인수로 받은 n초 동안 잠자고, 마지막 메시지 출력.
def loiter(n): 
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n))
    return n * 10

def main():
    display('Script starting.')
    executor = futures.ProcessPoolExecutor(max_workers=3)
    # worker가 3개이므로 일단은 loiter(0), loiter(1), loiter(2)만 먼저 실행
    results = executor.map(loiter, range(5)) 
    display('results:', results)
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))
    
main()


In [16]:
!python demo_executor_map_pr.py

[17:33:06] Script starting.
[17:33:06] results: <generator object _chain_from_iterable_of_lists at 0x7f0e27fb6c78>
[17:33:06] Waiting for individual results:
[17:33:06] loiter(0): doing nothing for 0s...
[17:33:06] loiter(0): done.
[17:33:06] 			loiter(3): doing nothing for 3s...
[17:33:06] 	loiter(1): doing nothing for 1s...
[17:33:06] 		loiter(2): doing nothing for 2s...
[17:33:06] result 0: 0
[17:33:07] 	loiter(1): done.
[17:33:07] 				loiter(4): doing nothing for 4s...
[17:33:07] result 1: 10
[17:33:08] 		loiter(2): done.
[17:33:08] result 2: 20
[17:33:09] 			loiter(3): done.
[17:33:09] result 3: 30
[17:33:11] 				loiter(4): done.
[17:33:11] result 4: 40


`Executor.map()`은 사용하기 쉽지만, 호출한 순서 그대로 결과를 반환하는 특징이 있다. 첫 번째 호출이 결과를 생성할 때까지 10초 걸리고 나머지 호출은 1초씩 걸린다면, `map()`이 반환한 제너레이터가 첫 번째 결과를 가져올 때까지 10초 걸린다. 더 진행하기 전에 모든 결과가 필요한 경우라면 이 특징은 문제가 되지 않지만, `submit()`한 순서와 상관없이 완료되는 대로 결과를 가져오는 게 좋은 경우도 있다. 이런 경우 `Executor.submit()` 메서드와 `futures.as_completed()` 함수를 함께 사용해야 한다.

## 17.5 진행 상황 출력하고 에러를 처리하며 내려받기

다양한 에러 조건의 처리를 테스트하기 위해 flags2 예제들을 살펴보자.


* `flags2_common.py` : 명령행 인수를 처리하고, 시간을 측정하고, 결과를 출력하는 main() 함수를 포함해서 공통으로 사용하는 함수와 설정을 담고 있다.
* `flags2_sequential.py` : 에러를 적절히 처리하며 진행 막대를 보여주는 순차 HTTP 클라이언트다.
* `flags2_threadpool.py` : futures.ThreadPoolExecutor에 기반해서 에러 처리와 진행 막대 통합을 보여주는 동시성 HTTP 클라이언트
* `flags2_asyncio.py` 스레드 예제와 기능상으로 동일하지만, asyncio와 aiohttp를 이용해서 구현한다.

여기서는 TQDM 패키지를 이용해서 진행 막대를 도입한다.

In [17]:
import time
from tqdm import tqdm
for i in tqdm(range(1000)):
    time.sleep(.01)

100%|██████████| 1000/1000 [00:10<00:00, 96.84it/s]


이 예제는 도움말도 지원한다.

In [18]:
!python flags2_threadpool.py -h

usage: flags2_threadpool.py [-h] [-a] [-e] [-l N] [-m CONCURRENT] [-s LABEL]
                            [-v]
                            [CC [CC ...]]

Download flags for country codes. Default: top 20 countries by population.

positional arguments:
  CC                    country code or 1st letter (eg. B for BA...BZ)

optional arguments:
  -h, --help            show this help message and exit
  -a, --all             get all available flags (AD to ZW)
  -e, --every           get flags for every possible code (AA...ZZ)
  -l N, --limit N       limit to N first codes
  -m CONCURRENT, --max_req CONCURRENT
                        maximum concurrent requests (default=30)
  -s LABEL, --server LABEL
                        Server to hit; one of DELAY, ERROR, LOCAL, REMOTE
                        (default=LOCAL)
  -v, --verbose         output detailed progress info


여기서 서버(-s)는 Nginx 구현이 잘 안되어서... 걍 github에 올려서 쓰자.

https://github.com/lih0905/lih0905.github.io/raw/master/flags/ad/ad.gif

각각의 스크립트는 인구가 많은 20개 나라의 국기를 가져온다.

In [19]:
!python flags2_sequential.py

REMOTE site: https://github.com/lih0905/lih0905.github.io/raw/master/flags
Searching for 20 flags: from BD to VN
1 concurrent connection will be used.
100%|███████████████████████████████████████████| 20/20 [01:21<00:00,  3.89s/it]
--------------------
20 flags downloaded.
Elapsed time: 81.02s


국가 코드가 A, B, C로 시작하는 모든 나라의 국기 이미지를 가져오려면 다음과 같이 한다.

In [20]:
!python flags2_threadpool.py 

REMOTE site: https://github.com/lih0905/lih0905.github.io/raw/master/flags
Searching for 20 flags: from BD to VN
20 concurrent connections will be used.
100%|███████████████████████████████████████████| 20/20 [00:02<00:00,  1.62it/s]
Segmentation fault (core dumped)


또 에러가 난다...

국가 코드를 선flags2_threadpool.py게 가져올 국기 이미지의 수를 `-l/--limit` 옵션으로 제한할 수 있다.

In [24]:
!python flags2_asyncio.py -al 100 -m 100

REMOTE site: https://github.com/lih0905/lih0905.github.io/raw/master/flags
Searching for 100 flags: from AD to LK
100 concurrent connections will be used.
100%|█████████████████████████████████████████| 100/100 [00:11<00:00,  8.73it/s]
Segmentation fault (core dumped)


### 17.5.1 flags2 예제에서의 에러 처리

이 스크립트들에서는 실제로 내려받는 `download_one()` 함수가 404 에러를 처리한다. 그 외 다른 에러는 위로 전달되어 `download_many()` 함수가 처리한다.

순차 코드부터 분석하자.

```python
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = requests.get(url)
    if resp.status_code != 200:  # 200 이외의 HTTP 코드에 대해 예외 발생시킴
        resp.raise_for_status()
    return resp.content


def download_one(cc, base_url, verbose=False):
    try:
        image = get_flag(base_url, cc)
    except requests.exceptions.HTTPError as exc:  # 404코드만 처리
        res = exc.response
        if res.status_code == 404:
            status = HTTPStatus.not_found  
            msg = 'not found'
        else:  # 그외엔 다시 HTTPError 예외를 발생시켜서 전달한다.
            raise
    else:
        save_flag(image, cc.lower() + '.gif')
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose:  # 상세 모드
        print(cc, msg)

    return Result(status, cc)  
# Result namedtuple은 status 필드에 HTTPStatus.not_found나 HTTPStatus.ok 값을 가진다.

def download_many(cc_list, base_url, verbose, max_req):
    counter = collections.Counter()  # Status의 합계를 구한다.
    cc_iter = sorted(cc_list)  # 정렬해서 보관
    if not verbose:
        cc_iter = tqdm.tqdm(cc_iter)  
    for cc in cc_iter:  
        try:
            res = download_one(cc, base_url, verbose)  # 국기 이미지 다운로드 시도
        except requests.exceptions.HTTPError as exc:  # 404 외 HTTP 에러 처리
            error_msg = 'HTTP error {res.status_code} - {res.reason}'
            error_msg = error_msg.format(res=exc.response)
        except requests.exceptions.ConnectionError as exc:  # 다른 네트워크 관련 처리
            error_msg = 'Connection error'
        else:  # 예외가 없다면 HTTPStatus.status 가져옴
            error_msg = '리
            status = res.status

        if error_msg:
            status = HTTPStatus.error  # 에러가 발생했다면 에러 저장
        counter[status] += 1  # 카운터 증가
        if verbose and error_msg: # 상세 메시지 모드에서 국가 코드에 대한 에러메시지 출력
            print('*** Error for {}: {}'.format(cc, error_msg))

    return counter  # counter를 반환
```

이 코드를 리팩토링해서 스레드 버전으로 만든 flags2_threadpool.py를 살펴보자.

### 17.5.2 futures.as_completed() 사용하기

각각의 요청마다 TQDM 진행 막대를 표시하고 에러 처리를 하기 위해 flags2_threadpool.py 스크립트는 `futures.as_completed()` 함수와 함께 `futures.ThreadPoolExecutor` 클래스를 사용한다.

```python
import collections
from concurrent import futures

import requests
import tqdm  

from flags2_common import main, HTTPStatus  
from flags2_sequential import download_one  # 재사용

DEFAULT_CONCUR_REQ = 30  # 최대 동시 요청 수의 기본값
MAX_CONCUR_REQ = 1000  # 동시 요청 수 제한


def download_many(cc_list, base_url, verbose, concur_req):
    counter = collections.Counter()
    with futures.ThreadPoolExecutor(max_workers=concur_req) as executor:  # 필요한 스레드만 생성
        to_do_map = {}  # 각각의 국가 코드에 Future 객체(하나의 내려받기 작업)를 매핑
        for cc in sorted(cc_list):  
            future = executor.submit(download_one,
                            cc, base_url, verbose)  # 하나의 콜러블의 실행을 스케줄링하며 Future 객체 반환
            # 첫번째 인수는 콜러블이며, 나머지 인수는 이 콜러블에 전달된다.
            to_do_map[future] = cc  
        done_iter = futures.as_completed(to_do_map)  # 완료된 순서대로 Future 객체를 생성하는 반복자 반환
        if not verbose:
            done_iter = tqdm.tqdm(done_iter, total=len(cc_list))  # done_iter에 len() 메서드가 없으므로,
            # tqdm()에 total 인수로 예상 항목 수를 알려줘야 tqdm()이 예상 시간을 계산할 수 있다.
        for future in done_iter:  # 완료되는 순서대로 Future 객체 반환
            try:
                res = future.result()  # 콜러블이 반환한 값이나 예외 발생
            except requests.exceptions.HTTPError as exc: 
                error_msg = 'HTTP {res.status_code} - {res.reason}'
                error_msg = error_msg.format(res=exc.response)
            except requests.exceptions.ConnectionError as exc:
                error_msg = 'Connection error'
            else:
                error_msg = ''
                status = res.status

            if error_msg:
                status = HTTPStatus.error
            counter[status] += 1
            if verbose and error_msg:
                cc = to_do_map[future]  # 에러 메시지에 대한 정보를 가져온다.
                # Future가 임의의 순서대로 완료되더라도 Future 객체의 결과를 가져와서 처리하기 쉬워진다.
                print('*** Error for {}: {}'.format(cc, error_msg))

    return counter

```